# Introduction to prompt chaining with Mistral Large #

This notebook should work well with the Data Science 3.0 kernel in SageMaker Studio.

This notebook will be an introduction to prompt chaining, a powerful technique used to enhance the capabilities of language models. In this guide, we will explore how prompt chaining effectively leverages the outputs of one model as inputs for another, creating a cascade of information processing that refines and improves the final output.

Prompt chaining is particularly useful when working with complex tasks that require nuanced understanding or multiple steps of reasoning. By using the output from one model iteration as the input for the next, we can achieve higher quality completions. This process allows subsequent models to build on the initial responses, assess their quality, and suggest improvements or further details, leading to more accurate and refined results.

Throughout this notebook, we will delve into the specifics of implementing prompt chaining with the Mistral Large model, this process also works with Mistral's other models on Bedrock like Mistral 8x7b. We will cover examples and strategies to maximize the effectiveness of this technique, illustrating how it can be applied to various scenarios to enhance decision-making and response quality. Whether you're looking to improve customer interactions, enhance content generation, or solve complex problems, this guide will provide you with the tools and knowledge needed to effectively implement prompt chaining in your projects.

## Mistral Large 2 ##


Available regions: US-West-2

Model ID: mistral.mistral-large-2407-v1:0

Context Window : 128k

In [ ]:
import boto3
import json
import requests
from bs4 import BeautifulSoup

In [ ]:
DEFAULT_MODEL= "mistral.mistral-large-2407-v1:0"

In [ ]:
class LLM:
    def __init__(self, model_id):
        self.model_id = model_id
        self.bedrock = boto3.client(service_name="bedrock-runtime")
        
    def invoke(self, prompt, temperature=0.0, max_tokens=3000):
        body = json.dumps({
            "temperature": temperature,
            "max_tokens": max_tokens,
            "prompt": prompt, 
            "stop": ["</s>"]
        })
        response = self.bedrock.invoke_model(
            body=body, 
            modelId=self.model_id)

        response_body = json.loads(response.get("body").read())
        return response_body['outputs'][0]['text']
    
llm = LLM(DEFAULT_MODEL)

Here, we will use the Mistral Wikipedia page as the source material for our notebook.

In [ ]:
url = "https://en.wikipedia.org/wiki/Mistral_AI"
response = requests.get(url)
data = response.text

In [ ]:
content = soup.find('div', class_='mw-parser-output')

In [ ]:
text_parts = []
for element in content.find_all(['p', 'h2', 'h3', 'h4', 'h5', 'h6']):
    text_parts.append(element.get_text())

In [ ]:
text = ' '.join(text_parts)

In [ ]:
clean_text = ' '.join(text.split())

In [ ]:
print(clean_text)

In this scenario, we'll pretend we're part of a customer service organization tasked with making our documentation easy to understand and accessible. Our initial prompt is designed to guide the model using the Mistral Wikipedia page as our source material. It outlines specific tasks to effectively create a comprehensive customer support resource. There are a lot of tasks for the model to complete - this is where prompt chaining can support the quality of our completion. Instead of relying solely on the first completion, we will have Mistral Large assess the first ouput in order to make improvements. These tasks include generating a detailed FAQ section, providing a concise summary, identifying key terms and notable figures, highlighting major milestones, and extracting relevant quotations. 

In [ ]:
initial_prompt = f"""You are a customer support agent at Acme Corp. Your role involves assisting customers by providing easy access to information about our company. Follow the instructions below for a successful outcome and use the article as context:

{clean_text}\n

Task 1: Create a detailed FAQ section based on the provided article. Generate 10-12 questions and answers that cover key aspects of the article, aimed at addressing common customer queries about our company. 

Task 2: Provide a high-level summary of the article, distilling the essential information into a concise overview.

Task 3: Identify and explain key terms, notable figures, and fundamental concepts from the article.

Task 4: Highlight major milestones in the development or history related to the article topic.

Task 5: Extract impactful quotations from the article and provide their context, if available.

Please ensure all information used in the FAQ, summary, key terms, milestones, and quotations is derived solely from the article provided, if you do not know the answer or how to proceed say so. Make sure you label each section so it is easy to read.
"""

In [ ]:
initial_completion = llm.invoke(initial_prompt, temperature=0.0, max_tokens=3000)
print("Initial completion:")
print(initial_completion)

Next, we will take the output from our first prompt and use it as input for the second step in our process. In this step, we include the source article as a reference to ensure that the feedback provided by the model is contextually accurate and valuable. The primary goal of this second step is to critically assess the initial output and provide necessary feedback or enhancements to add value and improve the overall quality of the information.

In [ ]:
assessment_prompt = f"""Given the following source article and generated content from it, please assess the quality of the output. Consider the accuracy, relevance, and comprehensiveness of the information provided in generated content. Highlight any discrepancies, inaccuracies, or missing details that should be addressed. Provide detailed feedback on how the generated summaries could be improved.

Source article:
{clean_text}

Generated content:
{step_1}

Your task is to:
1. Evaluate if the sections and text from the generated content information effectively adhere to the source article.
2. Check if the FAQs and other sections address likely customer queries effectively and whether the answers are correctly sourced from the article.
3. Suggest specific improvements or additions to provide comprehensive and useful information to the readers.

Please provide your feedback and suggestions in a structured format similar to the format found in the generated content. All feedback should be for making the content from the generated content better and should only be based on the source article.
"""
assessment_grade = llm.invoke(assessment_prompt, temperature=0.0, max_tokens=3000)
print("Assessment:")
print(assessment_grade)

In this phase of our notebook, we evaluate the initial outputs from the Mistral Wikipedia page, focusing on their accuracy, relevance, and comprehensiveness. This assessment addresses several sections including FAQs, a summary, key terms, milestones, and quotations. We identify areas that need improvement, such as expanding technical explanations, enriching biographical details, and updating timelines. The next step involves using the feedback to refine the content. We input the source article, the initial generated content, and our feedback into the Mistral Large model. The goal is to implement the suggested changes effectively, ensuring the final output is accurate, informative, and valuable for users.

In [ ]:
improvment_prompt =f"""Given the original article, the initia generated content from it, and the detailed assessment of their quality, your task is to implement the suggested improvements to enhance the accuracy, relevance, and comprehensiveness of the generated text.

Article:
{clean_text}

Generated content:
{step_1}

Assessment and Feedback:
{assessment_grade}

Instructions:
1. Revise all sections, if there is feedback specifically about it (Summary, FAQs, Key Terms, Milestones, Quotations) to better capture the essential points of the article, making sure to correct any inaccuracies noted in the feedback.
2. Ensure each section more effectively addresses the intended information needs, incorporating corrections and additional information as suggested in the assessment.
3. Maintain consistency with the information in the article and enhance the overall clarity and utility of all content sections.

Please provide the revised content, clearly marking any significant changes or additions only from the assessment provided to you. If you have additional feedback, please note this under 'Additional Feedback' as the last line in your response.
"""
final_product = llm.invoke(improvment_prompt, temperature=0.0, max_tokens=3000)
print("Final product:")
print(final_product)